In [1]:
import pandas as pd

In [24]:
df = pd.read_csv('data.csv')
print(df.columns)
df.head()

Index(['ID', 'Case Number', 'Date', 'Block', 'IUCR', 'Primary Type',
       'Description', 'Location Description', 'Arrest', 'Domestic', 'Beat',
       'District', 'Ward', 'Community Area', 'FBI Code', 'X Coordinate',
       'Y Coordinate', 'Year', 'Updated On', 'Latitude', 'Longitude',
       'Location'],
      dtype='object')


,ID,Case Number,Date,Block,IUCR,Primary Type,Description,Location Description,Arrest,Domestic,...,Ward,Community Area,FBI Code,X Coordinate,Y Coordinate,Year,Updated On,Latitude,Longitude,Location
0,24889,JD101272,01/02/2020 02:54:00 AM,072XX S SOUTH SHORE DR,110,HOMICIDE,FIRST DEGREE MURDER,APARTMENT,True,False,...,7.0,43.0,01A,1194878.0,1857803.0,2020,06/20/2020 03:48:45 PM,41.764728,-87.561272,"(41.764728045, -87.561272312)"
1,24890,JD101272,01/02/2020 03:17:00 AM,072XX S SOUTH SHORE DR,110,HOMICIDE,FIRST DEGREE MURDER,APARTMENT,True,False,...,7.0,43.0,01A,1194878.0,1857803.0,2020,06/20/2020 03:48:45 PM,41.764728,-87.561272,"(41.764728045, -87.561272312)"
2,24891,JD101694,01/02/2020 02:19:00 PM,069XX S MICHIGAN AVE,110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,6.0,69.0,01A,1178364.0,1858948.0,2020,06/19/2021 04:51:34 PM,41.768261,-87.621765,"(41.768260855, -87.621765291)"
3,24892,JD102066,01/02/2020 07:02:00 PM,082XX S DREXEL AVE,110,HOMICIDE,FIRST DEGREE MURDER,STREET,True,False,...,8.0,44.0,01A,1183667.0,1850610.0,2020,12/16/2021 03:47:24 PM,41.745258,-87.602587,"(41.745258504, -87.602587071)"
4,24893,JD103496,01/03/2020 08:57:00 PM,060XX S RACINE AVE,110,HOMICIDE,FIRST DEGREE MURDER,RETAIL STORE,True,False,...,16.0,68.0,01A,1169357.0,1864643.0,2020,06/20/2020 03:48:45 PM,41.784088,-87.654615,"(41.78408837, -87.654615414)"


In [25]:
import datetime
now = datetime.datetime(1999,1,1)
for i in range(df.shape[0]):
    # if Date is later than now, then update now
    tmp = datetime.datetime.strptime(df.Date[i], '%m/%d/%Y %H:%M:%S %p')
    if tmp > now:
        now = tmp
thirty_days_ago = now - datetime.timedelta(days=30)

In [34]:
df.iloc[480761,15] == None

False

In [41]:
from tqdm.notebook import tqdm_notebook
import numpy as np
rows,cols = 14,12
# four offences:THEFT,BATTERY,CRIMINAL DAMAGE,ASSAULT
lis = [[[[0]*4 for i in range(30)] for _ in range(cols)] for _ in range(rows)]
# get max and min for cols X coordinates and Y coordinates
x_min = df.min(axis=0)['X Coordinate']
x_max = df.max(axis=0)['X Coordinate']
y_min = df.min(axis=0)['Y Coordinate']
y_max = df.max(axis=0)['Y Coordinate']

x_gap = (x_max - x_min) / cols
y_gap = (y_max - y_min) / rows

# fill the data into list
for i in tqdm_notebook(range(df.shape[0])):
    # if Date is later than thirty_days_ago, then update lis
    tmp = datetime.datetime.strptime(df.Date[i], '%m/%d/%Y %H:%M:%S %p')
    if tmp > thirty_days_ago:
        # get the index of the list
        x_cord = df.iloc[i,15]
        y_cord = df.iloc[i,16]
        if np.isnan(x_cord) or np.isnan(y_cord):
            continue
        else:
            x_index = int((x_cord - x_min) / x_gap)
            y_index = int((y_cord - y_min) / y_gap)
        # tmp和now相差几天
        days = (now - tmp).days
        day_index = (30-1) - days
        # fill the lis based on the offence
        if df.iloc[i,5] == 'THEFT':
            lis[y_index][x_index][day_index][0] += 1
        elif df.iloc[i,5] == 'BATTERY':
            lis[y_index][x_index][day_index][1] += 1
        elif df.iloc[i,5] == 'CRIMINAL DAMAGE':
            lis[y_index][x_index][day_index][2] += 1
        elif df.iloc[i,5] == 'ASSAULT':
            lis[y_index][x_index][day_index][3] += 1
# save file
import pickle
a = np.array(lis)
print('file shape:',a.shape)
with open('Datasets/CHI_crime/pre.pkl', 'wb') as file:
    pickle.dump(a, file)

C:\Users\JPY\AppData\Local\Temp/ipykernel_10728/3201429136.py:7: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  x_min = df.min(axis=0)['X Coordinate']
C:\Users\JPY\AppData\Local\Temp/ipykernel_10728/3201429136.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  x_max = df.max(axis=0)['X Coordinate']
C:\Users\JPY\AppData\Local\Temp/ipykernel_10728/3201429136.py:9: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  y_min = df.min(axis=0)['Y Coordinate']
C:\Users\JPY\AppData\Local\Temp/ipykerne


file shape: (14, 12, 30, 4)
